In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [0]:
%tensorflow_version 2.x
import tensorflow as tf

In [0]:
text = open('shakespeare.txt', 'r').read()

In [5]:
print(text[:200])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
 


In [0]:
vocab = sorted(set(text))

In [7]:
vocab[:12]

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0']

In [8]:
len(vocab)

84

In [0]:
# Text Processing
# Vectorize the text, Encode the dictionary

In [10]:
for pair in enumerate(vocab):
    print(pair)

(0, '\n')
(1, ' ')
(2, '!')
(3, '"')
(4, '&')
(5, "'")
(6, '(')
(7, ')')
(8, ',')
(9, '-')
(10, '.')
(11, '0')
(12, '1')
(13, '2')
(14, '3')
(15, '4')
(16, '5')
(17, '6')
(18, '7')
(19, '8')
(20, '9')
(21, ':')
(22, ';')
(23, '<')
(24, '>')
(25, '?')
(26, 'A')
(27, 'B')
(28, 'C')
(29, 'D')
(30, 'E')
(31, 'F')
(32, 'G')
(33, 'H')
(34, 'I')
(35, 'J')
(36, 'K')
(37, 'L')
(38, 'M')
(39, 'N')
(40, 'O')
(41, 'P')
(42, 'Q')
(43, 'R')
(44, 'S')
(45, 'T')
(46, 'U')
(47, 'V')
(48, 'W')
(49, 'X')
(50, 'Y')
(51, 'Z')
(52, '[')
(53, ']')
(54, '_')
(55, '`')
(56, 'a')
(57, 'b')
(58, 'c')
(59, 'd')
(60, 'e')
(61, 'f')
(62, 'g')
(63, 'h')
(64, 'i')
(65, 'j')
(66, 'k')
(67, 'l')
(68, 'm')
(69, 'n')
(70, 'o')
(71, 'p')
(72, 'q')
(73, 'r')
(74, 's')
(75, 't')
(76, 'u')
(77, 'v')
(78, 'w')
(79, 'x')
(80, 'y')
(81, 'z')
(82, '|')
(83, '}')


In [0]:
char_to_ind = {char:ind for ind,char in enumerate(vocab)}
# created a dictionary for character to index

In [12]:
char_to_ind['A']

26

In [0]:
ind_to_char = np.array(vocab)
# created an array for index to character

In [14]:
ind_to_char[26]

'A'

In [0]:
encoded_text = np.array([char_to_ind[c] for c in text])
# Encode all the text

In [16]:
encoded_text

array([ 0,  1,  1, ..., 30, 39, 29])

In [17]:
encoded_text.shape
# 5.5 million character

(5445609,)

In [0]:
# Batch Creation

In [0]:
sequence_len = 120 # for learning the pattern of 3 lines

In [0]:
total_number_sequence = len(text)//(sequence_len+1)

In [21]:
total_number_sequence

45005

In [0]:
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

In [23]:
type(char_dataset)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [0]:
sequences = char_dataset.batch(sequence_len+1, drop_remainder=True)

In [0]:
def create_sequence_target(seq):
    input_txt = seq[:-1] # Hell
    target_text = seq[1:] # ello
    return input_txt, target_text

In [0]:
dataset = sequences.map(create_sequence_target)

In [0]:
batch_size= 128

In [0]:
buffer_size = 10000

dataset = dataset.shuffle(buffer_size=buffer_size).batch(batch_size=batch_size, drop_remainder=True)

In [29]:
dataset

<BatchDataset shapes: ((128, 120), (128, 120)), types: (tf.int64, tf.int64)>

In [0]:
# Model

In [0]:
vocab_size = len(vocab)

In [32]:
vocab_size

84

In [0]:
embedding_dimension = 64

In [0]:
rnn_neurons = 1026

In [0]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [36]:
help(sparse_categorical_crossentropy)

Help on function sparse_categorical_crossentropy in module tensorflow.python.keras.losses:

sparse_categorical_crossentropy(y_true, y_pred, from_logits=False, axis=-1)
    Computes the sparse categorical crossentropy loss.
    
    Usage:
    
    >>> y_true = [1, 2]
    >>> y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]
    >>> loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)
    >>> assert loss.shape == (2,)
    >>> loss.numpy()
    array([0.0513, 2.303], dtype=float32)
    
    Args:
      y_true: Ground truth values.
      y_pred: The predicted values.
      from_logits: Whether `y_pred` is expected to be a logits tensor. By default,
        we assume that `y_pred` encodes a probability distribution.
      axis: (Optional) Defaults to -1. The dimension along which the entropy is
        computed.
    
    Returns:
      Sparse categorical crossentropy loss value.



In [0]:
def sparse_cat_loss(y_true, y_pred):
    return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense

In [0]:
def  create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim, batch_input_shape=[batch_size,None]))
    model.add(GRU(rnn_neurons, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'))
    model.add(Dense(vocab_size))
    model.compile(optimizer='adam', loss=sparse_cat_loss)
    
    return model

In [0]:
model = create_model(vocab_size=vocab_size, embed_dim=embedding_dimension, batch_size=batch_size, rnn_neurons=rnn_neurons)

In [41]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 64)           5376      
_________________________________________________________________
gru (GRU)                    (128, None, 1026)         3361176   
_________________________________________________________________
dense (Dense)                (128, None, 84)           86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Make sure you run on GPU

In [0]:
for input_example_batch, target_example_batch in dataset.take(1):
  example_batch_predictions = model(input_example_batch)

In [0]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)

In [0]:
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

In [53]:
ind_to_char[sampled_indices]

array(['Z', '3', 'v', 'g', 'p', 'z', '_', 'X', '4', 'o', ' ', '!', 'G',
       'q', 'X', 'F', 'M', 'G', 'f', 'G', '7', 'f', 'n', '2', 'W', 'W',
       'S', '(', 'K', '!', '4', ')', '0', 'a', '&', 'X', 'j', 'w', '7',
       'N', 'N', 'H', '}', '[', 'p', 'B', '9', "'", '?', 'x', 'D', ' ',
       'D', 'P', '|', "'", 'h', 'T', '`', ';', 's', ':', 'W', '-', 'E',
       ' ', 'M', 'P', 'W', 'd', ',', 'a', 'T', 'h', 'p', 'U', 'k', 'N',
       '3', '.', 't', ',', 'O', 'w', 'D', 'T', '2', '\n', '<', '8', 's',
       ';', 'v', 'l', 'W', '5', '(', 'a', 'g', '_', 'o', 'N', '5', 'H',
       'J', 'L', 'P', 'H', 'N', 'Z', 'k', 'v', 'r', '<', ']', 'u', '1',
       'e', '!', 'K'], dtype='<U1')

In [0]:
epochs = 30

In [0]:
model.fit(dataset, epochs=epochs)

In [0]:
from tensorflow.keras.models import load_model

In [0]:
model = create_model(vocab_size, embedding_dimension, rnn_neurons, batch_size=1)
model.load_weights('shakespeare_gen.h5')
model.build(input_shape=tf.TensorShape([1, None]))

In [0]:
def generate_text(model, start_seed, gen_size=500, temp=1.0):
  num_generate = gen_size
  input_eval = [char_to_ind[s] for s in start_seed]
  input_eval = tf.expand_dims(input_eval, 0)

  text_generated = []
  temprature = temp

  model.reset_states()

  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)

    predictions = predictions/temprature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(ind_to_char[predicted_id])
  return start_seed+ "".join(text_generated)

In [61]:
print(generate_text(model, "JULIET", gen_size=1000))

JULIETTUCHATUS. What a beast thrust thy whore, As bring this supper this-
    And keep your queens, with Jueateral alive,
    I might not be as present up mine hand;
    If now, we now to your own scope my nurse,
    No more enjoin'd, methinks, which lims of power,
    Scare philosoproniculce obeys, sil, takes his hope
  PROTEUS. Ay, eas no further, though at my content!
    Some call the torch because what keeping Romeo
  This is the French exclaim'd.
  Pedro. Who should teath have merited such a scorn of death
    That I desiroured all together?
    Commend me to a word; or what is well to-morrow, what they say,
    The roty retreat. Am, what anks I would
    Prithee come in so empty? on Titinius!
    The Roman comfort, sir, to do you know,
    And whatso 'tis black, like hims
    As my reading in her beautiful
    Or less re.bean without a garland; nor no man burns
    hits; for if the base course of covert to the Tower, Master Bardeas not Lucius for you, frame
    To grial ingorate